In [0]:
# ! pip install -q kaggle

In [0]:
# from google.colab import files
# files.upload()

In [0]:
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list

In [0]:
# ! kaggle competitions download -c 'titanic'

In [0]:
import pandas as pd
import numpy as np
import featuretools as ft
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import math
import random
from sklearn import svm

In [0]:
TRAIN = pd.read_csv('/content/drive/My Drive/Machine Learning/Deep Feature Synthesis/train.csv')
TEST = pd.read_csv('/content/drive/My Drive/Machine Learning/Deep Feature Synthesis/test.csv')

In [0]:
def Preprocessing(DATASET):
    DATASET = DATASET.fillna(DATASET.mean())
    DATASET = DATASET.drop(columns = ['Cabin', 'Name', 'Ticket'])
    DATASET = DATASET.dropna()
    DATASET['Sex'].replace(['female', 'male'], [0, 1], inplace = True)
    DATASET['Embarked'].replace(['S','C', 'Q'], [0, 1, 2], inplace = True)
    print(DATASET.isnull().sum())
    print(DATASET.shape)

    return DATASET

In [8]:
TRAIN_PROCESSED = Preprocessing(TRAIN)

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64
(889, 9)


In [9]:
TEST_PROCESSED = Preprocessing(TEST)

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64
(418, 8)


In [0]:
Y_TRAIN = TRAIN_PROCESSED['Survived']
X_TRAIN = TRAIN_PROCESSED.drop(columns = ['Survived'])

In [0]:
def DFS(DATASET, DEPTH, ID):
    ES = ft.EntitySet()
    ES = ES.entity_from_dataframe(entity_id = 'Customers',
                                  dataframe = DATASET,
                                  index = 'PassengerId',
                                  variable_types = {'Pclass' : ft.variable_types.Categorical,
                                                    'Embarked' : ft.variable_types.Categorical})
    
    ES = ES.normalize_entity(base_entity_id = 'Customers',
                             new_entity_id = 'Trip',
                             index = 'PassengerId',
                             additional_variables=['Pclass', 'Fare', 'Embarked'])
    
    FEATURE_MATRIX, FEATURE_DEF = ft.dfs(entityset = ES,
                                         target_entity = str(ID),
                                         agg_primitives=["mean", "sum", "mode"],
                                         max_depth = DEPTH)

    return FEATURE_MATRIX, FEATURE_DEF

In [12]:
TRIP_FEATURE_MAT, TRIP_FEATURE_DEF = DFS(X_TRAIN, 2, 'Trip')
CUSTOMER_FEATURE_MAT, CUSTOMER_FEATURE_DEF = DFS(X_TRAIN, 2, 'Customers')

print(CUSTOMER_FEATURE_MAT.shape)
print(TRIP_FEATURE_MAT.shape)

(889, 15)
(889, 11)


In [0]:
def Merge_Data(DATASET_1, DATASET_2):
    DB = DATASET_1.join(DATASET_2)
    DB = DB.dropna(axis='columns')

    return DB

In [0]:
FINAL_TRAIN = Merge_Data(CUSTOMER_FEATURE_MAT, TRIP_FEATURE_MAT)

X_TRAINING, X_TESTING, Y_TRAINING, Y_TESTING = train_test_split(FINAL_TRAIN, Y_TRAIN, test_size = 0.33, random_state = 42)

In [0]:
# TRIP_FEATURE_MAT, TRIP_FEATURE_DEF = DFS(TEST_PROCESSED, 2, 'Trip')
# CUSTOMER_FEATURE_MAT, CUSTOMER_FEATURE_DEF = DFS(TEST_PROCESSED, 2, 'Customers')
# FINAL_TEST = Merge_Data(CUSTOMER_FEATURE_MAT, TRIP_FEATURE_MAT)

In [0]:
# Variables
POPULATION = 200
MUTATION_RATE = 0.1
NUM_FEATURES = 20

In [0]:
def MakePrediction(X, Y, i):
    CLF = svm.SVC()
    CLF.fit(X, Y)

    ACCURACY = CLF.score(X_TESTING[i], Y_TESTING)
    # PREDICTION = CLF.predict(X_TESTING[0:10])
    # REAL_VAL = Y_TESTING[0:10]
    # print('Prediction:', str(PREDICTION), 'Actual:', str(REAL_VAL.values))
    # print(CLF.score(X_TESTING, Y_TESTING))

    # FINAL_PREDICTION = CLF.predict(FINAL_TEST)
    # SUBMISSION = pd.DataFrame({'PassengerId':TEST['PassengerId'],'Survived':FINAL_PREDICTION})
    # SUBMISSION.to_csv('Titanic.csv', index=False)

    return ACCURACY

In [0]:
def CreatePopulation():
    LIST_POPULATION = []

    for i in range(POPULATION):
        FEATURES = np.random.choice(list(FINAL_TRAIN.columns.values), NUM_FEATURES).tolist()
        LIST_POPULATION.append(FEATURES)

    return LIST_POPULATION

In [0]:
def Natural_Selection():
    MATING_POOL = []
    FITNESS_POOL = []
    
    for i in LIST_POPULATION:
        X = X_TRAINING[i]
        Y = Y_TRAINING
        FITNESS_SCORE = MakePrediction(X, Y, i)
        FITNESS_POOL.append(FITNESS_SCORE)
        
        for j in range(math.floor(FITNESS_SCORE * 100 ** 2)):
            MATING_POOL.append(i) 

    FITTEST_INDEX = FITNESS_POOL.index(max(FITNESS_POOL))
    FITTEST_SCORE = max(FITNESS_POOL)
    FITTEST_CHILD = MATING_POOL[FITTEST_INDEX]

    print('Fittest Score :', FITTEST_SCORE)
    print('Fittest Child :', FITTEST_CHILD)
    
    return MATING_POOL, FITTEST_SCORE, FITTEST_CHILD

In [0]:
def Generate():
    for i in range(POPULATION):
        A = math.floor(random.randint(0, len(MATING_POOL) - 1))
        B = math.floor(random.randint(0, len(MATING_POOL) - 1))
        PARTNER_A = MATING_POOL[A]
        PARTNER_B = MATING_POOL[B]
        CHILD = CrossOver(PARTNER_A, PARTNER_B)
        CHILD = Mutation(CHILD)
        LIST_POPULATION[i] = CHILD

    return LIST_POPULATION

In [0]:
def CrossOver(PARTNER_A, PARTNER_B):
    CHILD = []

    MIDPOINT = math.floor(random.randint(0, (NUM_FEATURES - 1)))

    for i in range(len(PARTNER_A)):
        if i > MIDPOINT:
            CHILD.append(PARTNER_A[i])
        else:
            CHILD.append(PARTNER_B[i])

    return CHILD

In [0]:
def Mutation(CHILD):

    for i in range(NUM_FEATURES):
        if random.uniform(0, 1) < MUTATION_RATE:
            CHILD[i] = np.random.choice(list(FINAL_TRAIN.columns.values), 1).item()

    return CHILD

In [23]:
if __name__ == '__main__':
    LIST_POPULATION = CreatePopulation()
    FITTEST_SCORE = 0
    POOL = []
    COUNT = 0

    for COUNT in range(50):
    # while FITTEST_SCORE < 0.85:
        # COUNT = COUNT + 1
        print('ITERATION', COUNT)
        MATING_POOL, FITTEST_SCORE, FITTEST_CHILD = Natural_Selection()
        LIST_POPULATION = Generate()
        POOL.append(FITTEST_SCORE)

    print('The Fittest Score is', max(POOL))
    # print('The Fittest Child is', FITTEST_CHILD)

ITERATION 0
Fittest Score : 0.7585034013605442
Fittest Child : ['Trip.Pclass', 'Trip.Embarked', 'Trip.MEAN(Customers.Sex)', 'MEAN(Customers.Age)', 'Trip.MEAN(Customers.Parch)', 'Trip.Fare', 'Trip.MEAN(Customers.Sex)', 'Trip.MEAN(Customers.Sex)', 'Age', 'Trip.Embarked', 'Trip.SUM(Customers.Parch)', 'MEAN(Customers.Sex)', 'Trip.MEAN(Customers.Sex)', 'SUM(Customers.Sex)', 'SUM(Customers.Parch)', 'Trip.Embarked', 'Trip.SUM(Customers.Sex)', 'MEAN(Customers.SibSp)', 'Trip.MEAN(Customers.Age)', 'MEAN(Customers.Parch)']
ITERATION 1
Fittest Score : 0.8095238095238095
Fittest Child : ['Embarked', 'Age', 'SUM(Customers.Parch)', 'Trip.Pclass', 'MEAN(Customers.Parch)', 'MEAN(Customers.Age)', 'Trip.SUM(Customers.Age)', 'SUM(Customers.Parch)', 'Age', 'MEAN(Customers.SibSp)', 'Trip.Embarked', 'Sex', 'Trip.Fare', 'Trip.SUM(Customers.SibSp)', 'Sex', 'MEAN(Customers.SibSp)', 'Trip.SUM(Customers.Age)', 'Embarked', 'SUM(Customers.Age)', 'Trip.Embarked']
ITERATION 2
Fittest Score : 0.8095238095238095
Fittes